In [1]:
from torch.utils.data import Dataset

import numpy as np
import torch 
import os
import rasterio
import torch.nn.functional as F
from torchvision import transforms

mean = [1315.1941, 1320.8625, 1322.4388, 1312.7904, 1321.5713, 1331.3823, 1326.7014, 1314.1525, 1315.7151, 1313.9711]
std = [912.0858, 915.2389, 918.5796, 915.2799, 919.2444, 922.5997, 921.9182, 920.0427, 917.4285, 913.5229]
def pad_to_stride(tensor, stride):
    _, h, w = tensor.shape
    pad_h = (stride - h % stride) 
    pad_w = (stride - w % stride)
    padding = (0, pad_w, 0, pad_h)  # (w_left, w_right, h_top, h_bottom)
    return torch.nn.functional.pad(tensor, padding,mode='reflect')
def prepare(path_images,path_masks,files):
    output_stride = 256
    result=None
    result_m=None
    
    
    for i in files:
        with rasterio.open(f'{path_images}/{i}') as src:
            image = src.read().astype(np.float32)  # Read image
            image = torch.tensor(image)
    
            image = pad_to_stride(image, output_stride)
            h_splits = image.shape[1] // output_stride
            w_splits = image.shape[2] // output_stride
            tensor_split = image.unfold(1, output_stride, output_stride).unfold(2, output_stride, output_stride)
            tensor_split = tensor_split.contiguous().view(-1, image.shape[0], output_stride, output_stride)
        with rasterio.open(f'{path_masks}/{i}') as src:
            image_m = src.read().astype(np.float32)  # Read image
            image_m = torch.tensor(image_m)
    
            image_m = pad_to_stride(image_m, output_stride)
            h_splits_m = image_m.shape[1] // output_stride
            w_splits_m = image_m.shape[2] // output_stride
            tensor_split_m = image_m.unfold(1, output_stride, output_stride).unfold(2, output_stride, output_stride)
            tensor_split_m = tensor_split_m.contiguous().view(-1, image_m.shape[0], output_stride, output_stride)
    
        # Model evaluation
        if result is not None:
            result=torch.cat((result, tensor_split), dim=0)
            result_m=torch.cat((result_m, tensor_split_m), dim=0)
        else:
            result=tensor_split
            result_m=tensor_split_m
    return result,result_m

In [2]:
# model

In [3]:

from typing import Any

import timm
import torch
from timm.models import ResNet
from torchvision.models._api import Weights, WeightsEnum
import torchgeo.transforms.transforms as T






/home/user1/environments/hack/lib/python3.10/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [4]:
import os
from torchvision import transforms
import torch
import torch.nn as nn
import rasterio
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from typing import List, Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from dataset_create import WaterDataset,prepare,pad_to_stride
from seg_model import seg_model_consturct,get_loss

In [6]:
model = seg_model_consturct(
    model= 'deeplabv3+',
    backbone= 'resnet50',
    weights= None,
    in_channels=10,
    num_classes=1,
)
model.load_state_dict(torch.load("best_focal_e3.pth"))

/tmp/ipykernel_202086/2450901176.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_focal_e3.pth"))


<All keys matched successfully>

In [9]:
device='cuda'

In [14]:
import os
output_stride = 256
model.to('cuda')
for i in os.listdir('../train/images/'):
    print(i)
    with rasterio.open(f'../train/images/{i}') as src:
        image = src.read().astype(np.float32)  # Read image
        print(image.shape)
        image = torch.tensor(image)
        size_out = image.size()

        # Pad to make dimensions divisible by 16
        image = pad_to_stride(image, output_stride)
        print(image.shape)
        h_splits = image.shape[1] // output_stride
        w_splits = image.shape[2] // output_stride
        tensor_split = image.unfold(1, output_stride, output_stride).unfold(2, output_stride, output_stride)
        tensor_split = tensor_split.contiguous().view(-1, image.shape[0], output_stride, output_stride)
        print(tensor_split.shape)
        meta = src.meta

    # Model evaluation
    model.eval()
    sig = torch.nn.Sigmoid()
    outputs = []

    for tensor in tensor_split:
        with torch.no_grad():
            output = sig(model(tensor.to(device).unsqueeze(0))).cpu()
            outputs.append(output)
    output_rows = []
    for h in range(h_splits):
        row_outputs = outputs[h * w_splits : (h + 1) * w_splits]
        output_rows.append(torch.cat(row_outputs, dim=-1))  

    final_output = torch.cat(output_rows, dim=-2)


    meta['count'] = 1
    pred = (final_output.squeeze(0)<0.5).cpu().numpy().astype("int32")[0]
    pred = pred[:size_out[1], :size_out[2]]

    with rasterio.open(f'test_output/{i}', 'w', **meta) as fout:
        fout.write(pred, 1)
    print(pred.shape)

5.tif
(10, 2336, 3230)
torch.Size([10, 2560, 3328])
torch.Size([130, 10, 256, 256])
(2336, 3230)
6_2.tif
(10, 4971, 10632)
torch.Size([10, 5120, 10752])
torch.Size([840, 10, 256, 256])
(4971, 10632)
1.tif
(10, 10134, 10680)
torch.Size([10, 10240, 10752])
torch.Size([1680, 10, 256, 256])
(10134, 10680)
2.tif
(10, 2648, 6744)
torch.Size([10, 2816, 6912])
torch.Size([297, 10, 256, 256])
(2648, 6744)
9_2.tif
(10, 512, 512)
torch.Size([10, 768, 768])
torch.Size([9, 10, 256, 256])
(512, 512)
9_1.tif
(10, 512, 512)
torch.Size([10, 768, 768])
torch.Size([9, 10, 256, 256])
(512, 512)
6_1.tif
(10, 4971, 10632)
torch.Size([10, 5120, 10752])
torch.Size([840, 10, 256, 256])
(4971, 10632)
4.tif
(10, 4389, 6708)
torch.Size([10, 4608, 6912])
torch.Size([486, 10, 256, 256])
(4389, 6708)


In [11]:
mkdir test_output

In [15]:
!ls
# !rm -rf test_output/*

__pycache__			   new_train_loop-pretain-Copy1.ipynb
best_ce_e3.pth			   new_train_loop-pretain-Copy2.ipynb
best_focal.pth			   new_train_loop-pretain.ipynb
best_focal_e3.pth		   new_train_loop.ipynb
best_jaccard.pth		   save_result_copy-alan.ipynb
best_pretrain_deeplabv_50.pth	   seg_model.py
best_without.pth		   sentinel2_resnet50_mi_ms-da5413d2.pth
dataset_create.py		   test_output
new_train_loop-jaccard_pred.ipynb


In [ ]:
#!/bin/bash

OSM_PATH="train/osm"
MASKS_PATH="train/masks"
PREDS_PATH="test_output"
PRE_GT_PATH="9_1.tif"
POST_GT_PATH="9_2.tif"
PRE_PRED_PATH="9_1_out.tif"
POST_PRED_PATH="9_2_out.tif"

python3 calculate_metrics.py \
    --osm_path "$OSM_PATH" \
    --masks_path "$MASKS_PATH" \
    --preds_path "$PREDS_PATH" \
    --pre_gt_path "$PRE_GT_PATH" \
    --post_gt_path "$POST_GT_PATH" \
    --pre_pred_path "$PRE_PRED_PATH" \
    --post_pred_path "$POST_PRED_PATH"


In [ ]:
#!/bin/bash
OSM_PATH="train/osm"
MASKS_PATH="train/masks"
PREDS_PATH="test_output"
PRE_GT_PATH="9_1.tif"
POST_GT_PATH="9_2.tif"
PRE_PRED_PATH="9_1_out.tif"
POST_PRED_PATH="9_2_out.tif"

python3 calculate_metrics.py \
    --osm_path "train/osm" \
    --masks_path "train/masks" \
    --preds_path "test_output" \
    --pre_gt_path "9_1.tif" \
    --post_gt_path "9_2.tif" \
    --pre_pred_path "9_1_out.tif" \
    --post_pred_path "9_2_out.tif"


In [ ]:
python3 calculate_metrics.py \
     --osm_path "train/osm/9.geojson" \
     --masks_path "train/masks" \
     --preds_path "test_output" \
     --pre_gt_path "train/masks/9_1.tif" \
     --post_gt_path "train/masks/9_2.tif" \
     --pre_pred_path "test_output/9_1.tif" \
     --post_pred_path "test_output/9_2.tif"


example_preds

In [ ]:
python3 calculate_metrics.py \
     --osm_path "train/osm/9.geojson" \
     --masks_path "train/masks" \
     --preds_path "train/example_preds" \
     --pre_gt_path "train/masks/9_1.tif" \
     --post_gt_path "train/masks/9_2.tif" \
     --pre_pred_path "train/example_preds/9_1.tif" \
     --post_pred_path "train/example_preds/9_2.tif"



In [ ]:
python3 calculate_metrics.py \
     --osm_path "train/osm/9.geojson" \
     --masks_path "train/masks" \
     --preds_path "../test_output" \
     --pre_gt_path "train/masks/9_1.tif" \
     --post_gt_path "train/masks/9_2.tif" \
     --pre_pred_path "../test_output/9_1.tif" \
     --post_pred_path "../test_output/9_2.tif"


example_preds

In [ ]:
# pre_pred_path = "path_to_pre_pred_image.tif"
# post_pred_path = "path_to_post_pred_image.tif"
# pre_pred = infer_and_normalize(pre_pred_path)
# post_pred = infer_and_normalize(post_pred_path)

# # Открытие данных Ground Truth (GT)
# pre_gt_path = "path_to_pre_gt_image.tif"
# post_gt_path = "path_to_post_gt_image.tif"

# with rasterio.open(pre_gt_path) as src:
#     pre_gt = src.read(1)  # Читаем Ground Truth

# with rasterio.open(post_gt_path) as src:
#     post_gt = src.read(1)  # Читаем Ground Truth

# # Вычисление метрик
# masks_path = "train/masks"
# preds_path = "path_to_preds"
# osm_path = "train/osm"

# f1_water = calculate_f1_score(masks_path, preds_path)

# # Геоданные для расчета затопленных домов
# pre_height, pre_width = pre_gt.shape
# pre_cols, pre_rows = np.meshgrid(np.arange(pre_width), np.arange(pre_height))
# pre_x, pre_y = rasterio.transform.xy(src.transform, pre_rows, pre_cols) 
# pre_lons, pre_lats = np.array(pre_x), np.array(pre_y)

# post_height, post_width = post_gt.shape
# post_cols, post_rows = np.meshgrid(np.arange(post_width), np.arange(post_height))
# post_x, post_y = rasterio.transform.xy(src.transform, post_rows, post_cols) 
# post_lons, post_lats = np.array(post_x), np.array(post_y)

# pre_f1 = flooded_houses(osm_path, pre_lats, pre_lons, pre_pred, pre_gt)
# post_f1 = flooded_houses(osm_path, post_lats, post_lons, post_pred, post_gt)
# avg_f1_business = (pre_f1 + post_f1) / 2

# print(f"F1-Score: {(f1_water + avg_f1_business) / 2 :.3f}")